**Név: Veres Melitta-Kinga**

**Nr. matricol: 4447**

1. A **lab2/6. feladatot** módosítsuk a következő módon:

\-Hirdessünk **tranzakciót** a tárolt eljárásban: vagy minden beszúrást hajtson végre a rendszer, vagy egyiket sem!

\-Az adatok adatbázisban való meglétének ellenőrzését, az ID-k tárolását stb. oldjuk meg a lehető legkevesebb lekérdezés segítségével (használjunk lokális változókat)!

\-Minden beszúrás esetén **ellenőrizzük le**, hogy megtörtént-e a beszúrás. Ha nem, hiba(!)üzenet kiíratása mellett térjünk vissza -1-el a tárolt eljárásból (visszatérítési(!) érték).

\-**Fejezzük be minden esetben (pl. feltételes utasítások minden ágán) megfelelően a tranzakciót!**

In [40]:
CREATE OR ALTER PROCEDURE Kocsmabeszuras
    @pKocsmaNev VARCHAR(20),
    @pCim VARCHAR(30),
    @pNyitasDatuma DATE,
    @pBaratNev VARCHAR(30),
    @pBaratEmail VARCHAR(20),
    @pItalTipNev VARCHAR(10),
    @pItalNev VARCHAR(20),
    @pAr REAL
AS
BEGIN
    SET NOCOUNT ON;
    DECLARE @vUjKocsmaID INT;
    DECLARE @vUjItalID INT;
    DECLARE @vUjItalTipID INT;
    DECLARE @vBeszurasSikeres BIT = 1;

    BEGIN TRY
        BEGIN TRANSACTION;

        IF EXISTS (SELECT Nev FROM Kocsmak WHERE Nev = @pKocsmaNev)
        BEGIN
            RAISERROR('A megadott kocsma már létezik az adatbázisban.', 8, 1);
            SET @vBeszurasSikeres = 0;
        END
        ELSE
        BEGIN
            
            IF NOT EXISTS (SELECT Nev FROM Baratok WHERE Nev = @pBaratNev)
                INSERT INTO Baratok (Nev, Email) VALUES (@pBaratNev, @pBaratEmail);

            IF NOT EXISTS (SELECT TipusNev FROM ItalTipusok WHERE TipusNev = @pItalTipNev)
                INSERT INTO ItalTipusok (TipusNev) VALUES (@pItalTipNev);

            IF NOT EXISTS (SELECT Nev FROM Italok WHERE Nev = @pItalNev)
            BEGIN
                SET @vUjItalTipID = (SELECT TipusID FROM ItalTipusok WHERE TipusNev = @pItalTipNev);
                INSERT INTO Italok (Nev, TipusID) VALUES (@pItalNev, @vUjItalTipID);
            END

            
            INSERT INTO Kocsmak (Nev, Cim, NyitasDatuma) VALUES (@pKocsmaNev, @pCim, @pNyitasDatuma);

            
            SET @vUjKocsmaID = (SELECT KocsmaID FROM Kocsmak WHERE Nev = @pKocsmaNev);
            SET @vUjItalID = (SELECT ItalID FROM Italok WHERE Nev = @pItalNev);


            INSERT INTO Arak (Ar, KocsmaID, ItalID) VALUES (@pAr, @vUjKocsmaID, @vUjItalID);

        END
    END TRY
    BEGIN CATCH
        SET @vBeszurasSikeres = 0;
        SELECT
            ERROR_NUMBER() AS ErrNum,
            ERROR_MESSAGE() AS ErrMsg,
            ERROR_STATE() AS ErrState,
            ERROR_SEVERITY() AS ErrSeverity,
            ERROR_LINE() AS ErrLine,
            ERROR_PROCEDURE() AS ErrProcedure;
        ROLLBACK TRANSACTION;
    END CATCH;

    IF @vBeszurasSikeres = 1
        COMMIT TRANSACTION;
    ELSE
        ROLLBACK TRANSACTION;

    RETURN CASE WHEN @vBeszurasSikeres = 1 THEN 0 ELSE -1 END;
END

Commands completed successfully.

Batch execution time: 00:00:00.015

Total execution time: 00:00:00.015

In [5]:
GO
EXEC KocsmaBeszuras 
@pKocsmaNev='EE',
@pCim='Cim Hibrid',
@pNyitasDatuma='2023-11-10',
@pBaratNev='Vajk',
@pBaratEmail='vajk@yahoo.com',
@pItalTipNev='kave',
@pItalNev='espresso',
@pAr=10


Commands completed successfully.

Batch execution time: 00:00:00.043

Total execution time: 00:00:00.043

**2\. a.** **Szúrjuk be** <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">a Kedveltseg mezőt a Kocsmak táblába, ahol Kedveltseg-INT típusú mező, 0-5 közötti értékeket vehet fel, alapértelmezés szerinti értéke 0.</span>

In [38]:
CREATE OR ALTER PROCEDURE Kedveltseg
AS
BEGIN
    SET NOCOUNT ON;

    SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;

    BEGIN TRY
        BEGIN TRANSACTION;

        IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'Kocsmak' AND COLUMN_NAME = 'Kedveltseg')
        BEGIN
            ALTER TABLE [dbo].[Kocsmak]
            ADD Kedveltseg INT NOT NULL DEFAULT 0 CHECK (Kedveltseg >= 0 AND Kedveltseg <= 5);

            IF @@ERROR <> 0
            BEGIN 
                RAISERROR('Hiba történt az oszlop létrehozása során!', 8, 1);
                RETURN;
            END
        END

        UPDATE [dbo].[Kocsmak]
        SET Kedveltseg = 0;

        IF @@ERROR <> 0
        BEGIN 
            RAISERROR('Hiba történt a kedveltseg frissítése során!', 8, 1);
        END

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        SELECT
            ERROR_NUMBER()  AS ErrNum,
            ERROR_MESSAGE()  AS ErrMsg,
            ERROR_STATE() AS ErrState,
            ERROR_SEVERITY() AS ErrSeverity,
            ERROR_LINE() AS ErrLine,
            ERROR_PROCEDURE() AS ErrProcedure;

        ROLLBACK TRANSACTION;
    END CATCH;
END


Commands completed successfully.

Batch execution time: 00:00:00.004

Total execution time: 00:00:00.004

In [9]:
GO
EXEC Kedveltseg 

Commands completed successfully.

Batch execution time: 00:00:00.054

Total execution time: 00:00:00.054

2. **b.** Írjunk **tárolt eljárást**, mely minden kocsma esetén beállítja a Kedveltseg mező értékét annak megfelelően, hogy hány barátunk adott neki 2-nél nagyobb értékelést:

- Barátok száma \< = 4  **(0 is lehet!)** –\> Kedveltseg = 1;
- Barátok száma \> 4 és \<= 10  –\> Kedveltseg = 2;
- Barátok száma \> 10 és \<= 20  –\> Kedveltseg = 3;
- Barátok száma \> 20 és \<= 30  –\> Kedveltseg = 4;
- Barátok száma \> 30  –\> Kedveltseg = 5;

In [10]:
CREATE OR ALTER PROCEDURE Kedveltsegbeallitasa
AS
BEGIN
    SET NOCOUNT ON;

    SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;

    BEGIN TRY
        BEGIN TRANSACTION kedveltsegbeallitasa;

        DECLARE @KocsmaID INT, @BaratokSzama INT;

        DECLARE kocsmak_cursor CURSOR FOR
        SELECT KocsmaID, COUNT(*) AS BaratokSzama
        FROM Kedvencek
        WHERE Ertekeles >= 2
        GROUP BY KocsmaID;

        OPEN kocsmak_cursor;

        FETCH NEXT FROM kocsmak_cursor INTO @KocsmaID, @BaratokSzama;

        WHILE @@FETCH_STATUS = 0
        BEGIN
            UPDATE Kocsmak
            SET Kedveltseg = 
                CASE
                    WHEN @BaratokSzama <= 4 THEN 1
                    WHEN @BaratokSzama > 4 AND @BaratokSzama <= 10 THEN 2
                    WHEN @BaratokSzama > 10 AND @BaratokSzama <= 20 THEN 3
                    WHEN @BaratokSzama > 20 AND @BaratokSzama <= 30 THEN 4
                    WHEN @BaratokSzama > 30 THEN 5
                END
            WHERE KocsmaID = @KocsmaID;

            FETCH NEXT FROM kocsmak_cursor INTO @KocsmaID, @BaratokSzama;
        END

        CLOSE kocsmak_cursor;

        DEALLOCATE kocsmak_cursor;

        COMMIT TRANSACTION kedveltsegbeallitasa;
    END TRY
    BEGIN CATCH
        SELECT
            ERROR_NUMBER()  AS ErrNum,
            ERROR_MESSAGE()  AS ErrMsg,
            ERROR_STATE() AS ErrState,
            ERROR_SEVERITY() AS ErrSeverity,
            ERROR_LINE() AS ErrLine,
            ERROR_PROCEDURE() AS ErrProcedure;

        ROLLBACK TRANSACTION kedveltsegbeallitasa;
    END CATCH;
END


Commands completed successfully.

Batch execution time: 00:00:00.035

Total execution time: 00:00:00.035

In [11]:
GO
EXEC Kedveltsegbeallitasa

Commands completed successfully.

Batch execution time: 00:00:00.040

Total execution time: 00:00:00.040

<span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">3. Írjunk INSERT és DELETE triggert, mely a Kedvencek táblába való beszúrás illetve törlés esetén aktualizálja a Kocsmak tábla Kedveltseg mezőjének értékét! Fontos! Nem elfogadott megoldás: ha a triggeren belül egyszerűen csak meghívjuk a 2. feladatban megírt tárolt eljárást!</span>

In [44]:
/*mukodik egyszerre tobb sorra*/
CREATE TRIGGER tr_insert_delete_Kedvencek
ON Kedvencek
AFTER INSERT, DELETE
AS
BEGIN

    UPDATE Kocsmak
    SET Kedveltseg = 
        CASE
            WHEN (SELECT COUNT(*) FROM Kedvencek WHERE Kedvencek.KocsmaID = Kocsmak.KocsmaID AND Kedvencek.Ertekeles >= 2) <= 4 THEN 1
            WHEN (SELECT COUNT(*) FROM Kedvencek WHERE Kedvencek.KocsmaID = Kocsmak.KocsmaID AND Kedvencek.Ertekeles >= 2) > 4 AND (SELECT COUNT(*) FROM Kedvencek WHERE Kedvencek.KocsmaID = Kocsmak.KocsmaID AND Kedvencek.Ertekeles >= 2) <= 10 THEN 2
            WHEN (SELECT COUNT(*) FROM Kedvencek WHERE Kedvencek.KocsmaID = Kocsmak.KocsmaID AND Kedvencek.Ertekeles >= 2) > 10 AND (SELECT COUNT(*) FROM Kedvencek WHERE Kedvencek.KocsmaID = Kocsmak.KocsmaID AND Kedvencek.Ertekeles >= 2) <= 20 THEN 3
            WHEN (SELECT COUNT(*) FROM Kedvencek WHERE Kedvencek.KocsmaID = Kocsmak.KocsmaID AND Kedvencek.Ertekeles >= 2) > 20 AND (SELECT COUNT(*) FROM Kedvencek WHERE Kedvencek.KocsmaID = Kocsmak.KocsmaID AND Kedvencek.Ertekeles >= 2) <= 30 THEN 4
            WHEN (SELECT COUNT(*) FROM Kedvencek WHERE Kedvencek.KocsmaID = Kocsmak.KocsmaID AND Kedvencek.Ertekeles >= 2) > 30 THEN 5
        END
    FROM Kocsmak
    INNER JOIN (SELECT KocsmaID FROM INSERTED UNION SELECT KocsmaID FROM DELETED) AS Kedvencek ON Kocsmak.KocsmaID = Kedvencek.KocsmaID;
END;
GO


Commands completed successfully.

Batch execution time: 00:00:00.010

Total execution time: 00:00:00.010

4. **a.** Módositsuk az adatbázisunkat a következő módon:

- Keszitsünk egy Eladasok táblát mely a naponta eladott termékeket tartalmazza egy illető Kocsmában: ItalID:Int, KocsmaID:Int, Mennyiseg:Int , Egysegar: float, Datum:Date(aktuális dátum alapértelmezett értékkel) Az elsődleges kulcs legyen a KocsmaID, ItalID, Datum összetett kulcs valamint a KocsmaID és ItalID külső kulcsok.

In [45]:
CREATE TABLE Eladasok (
    KocsmaID INT,
    ItalID INT,
    Datum DATE DEFAULT GETDATE(),
    Mennyiseg INT,
    Egysegar FLOAT,
    PRIMARY KEY (KocsmaID, ItalID, Datum),
    FOREIGN KEY (KocsmaID) REFERENCES Kocsmak(KocsmaID),
    FOREIGN KEY (ItalID) REFERENCES Italok(ItalID)
);


Commands completed successfully.

Batch execution time: 00:00:00.011

Total execution time: 00:00:00.011

4. **b.**  <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">&nbsp;Írj tárolt eljárást mely felvezet az Eladások táblába. A tárolt eljárás paraméterei: @KocsmaNev, @ItalNev,&nbsp; @ItalTipusNev, @Mennyiseg, @EgysegAr. A tárolt eljárás ellenőrizze, a kocsma létezését,&nbsp; ha nem létezik hibát jelezve lépjen ki, valamint, hogy az eladott mennyiség pozitív. Ha az&nbsp;</span>  <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">@ItalNev</span>  <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">&nbsp;vagy a hozzá tartozó</span> <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">@ItalTipusNev típusú ital&nbsp;</span>  <span style="color: rgb(45, 59, 69); font-family: LatoWeb, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(255, 255, 255);">nem létezik kerüljon felvezetésre az Italok és ItalTipusok táblába valamint a megadott @EgysegAr-al az Arak táblába is. Abban az esetben ha létezik, frissítse az Arak.Ar mezőjét az uj értékkel ha az változott. Az eladás felvezetésekor az eladás Dátuma legyen az aktuális dátum. Végül frissítse a kocsma bevételét hozzáadva a @Mennyiseg * @EgysegAr erteket.</span>

In [56]:
CREATE PROCEDURE Eladastfelvezetniii
    @KocsmaNev VARCHAR(30),
    @ItalNev VARCHAR(30),
    @ItalTipusNev VARCHAR(30),
    @Mennyiseg INT,
    @EgysegAr FLOAT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @KocsmaID INT;
    SELECT @KocsmaID = KocsmaID FROM Kocsmak WHERE Nev = @KocsmaNev;

    IF @KocsmaID IS NULL
    BEGIN
        RAISERROR ('Hiba: A megadott kocsmának nincs bejegyzése!', 8, 1);
        RETURN;
    END

    IF @Mennyiseg <= 0
    BEGIN
        RAISERROR('Hiba: Az eladott mennyiségnek pozitívnak kell lennie!', 16, 1);
        RETURN;
    END

    DECLARE @ItalID INT;
    SELECT @ItalID = ItalID
    FROM Italok i
    JOIN ItalTipusok t ON i.TipusID = t.TipusID
    WHERE Nev = @ItalNev AND TipusNev = @ItalTipusNev;

    DECLARE @TipusID INT;
    SELECT @TipusID = TipusID FROM ItalTipusok WHERE TipusNev = @ItalTipusNev;

    IF @TipusID IS NULL
    BEGIN
        INSERT INTO ItalTipusok (TipusNev) VALUES (@ItalTipusNev);
        SELECT @TipusID = TipusID FROM ItalTipusok WHERE TipusNev=@ItalTipusNev;
    END

    IF @ItalID IS NULL
    BEGIN
        INSERT INTO Italok (Nev, TipusID) VALUES (@ItalNev, @TipusID);
        SELECT @ItalID = ItalID FROM Italok WHERE Nev=@ItalNev;
    END

    DECLARE @Ar FLOAT;
    SELECT @Ar = Ar FROM Arak WHERE ItalID = @ItalID;

    IF @Ar IS NULL
    BEGIN
        INSERT INTO Arak (ItalID, Ar, KocsmaID) VALUES (@ItalID, @EgysegAr, @KocsmaID);
    END
    ELSE
    BEGIN
        IF @EgysegAr <> (SELECT Ar FROM Arak WHERE ItalID = @ItalID)
        BEGIN
            UPDATE Arak SET Ar = @EgysegAr WHERE ItalID = @ItalID;
        END
    END

    IF @ItalID IS NOT NULL
    BEGIN
        INSERT INTO Eladasok (KocsmaID, ItalID, Mennyiseg, Egysegar, Datum)
        VALUES (@KocsmaID, @ItalID, @Mennyiseg, @EgysegAr, GETDATE());

        UPDATE Kocsmak
        SET Bevetel = Bevetel + (@Mennyiseg * @EgysegAr)
        WHERE KocsmaID = @KocsmaID;
    END
END;


Commands completed successfully.

Batch execution time: 00:00:00.016

Total execution time: 00:00:00.016

In [61]:
GO
EXEC Eladastfelvezetniii
@KocsmaNev='kocsma',
@ItalNev='Hugo',
@ItalTipusNev='pezsgo',
@Mennyiseg=-1,
@Egysegar=10;

Msg 50000, Level 8, State 1, Procedure Eladastfelvezetniii, Line 16
Hiba: A megadott kocsmának nincs bejegyzése!

Batch execution time: 00:00:00.019

Total execution time: 00:00:00.019